## Setup

In [1]:
from traveltimepy import Location, Transportation, Walking, Coordinates, TravelTimeSdk, Property, FullRange
# Full setup
import pandas as pd
import asyncio
from datetime import datetime
from dataclasses import dataclass

df_latlongs = pd.read_csv('latlongs.csv')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Functions

In [8]:
@dataclass
class StationGeography: 
    '''Simple way to store a station identifier (no tight definition around this) and a lat/long pair'''
    station_id: str     # Any identifier for the station
    latitude: float     
    longitude: float


async def time_filter_wrapper(sdk: TravelTimeSdk, origin:StationGeography, destination:StationGeography, transport_type:Transportation) -> int:
    """
    Wrapper to get TravelTimeSdk to work with code. Largely copied from the API documentation.
    This is set as async in Python because the function in the API requires it.

    Parameters:
        sdk - Instance of TravelTimeSdk with app_id and api_key, which you can get for free
        origin - Instance of StationGeography, this is the origin of the trip
        destination - Instnace of StationGeography, this is the destination of the trip
        transport_type - Could pass any instance of Transportation. 


    """
    locations = [
        Location(id=origin.station_id, coords=Coordinates(
            lat=origin.latitude, lng=origin.longitude)),
        Location(id=destination.station_id, coords=Coordinates(
            lat=destination.latitude, lng=destination.longitude))
    ]

    results = await sdk.time_filter_async(
        locations=locations,
        search_ids={
            origin.station_id: [destination.station_id]
        },
        departure_time=datetime.now(),
        travel_time=10800,  # max travel time (sec) allowed by the API, ensures that it doesn't return NULL, even if there is a valid path
        transportation=transport_type,  # I pass either Walking or Transportation(public_transit = "train")
        properties=[Property.TRAVEL_TIME],
        range=FullRange(enabled=False, max_results=3, width=600)
    )
    try:
        # The API is funky, so the extraction doesn't work sometimes
        travel_time = results[0].__dict__.get('locations')[0].properties[0].travel_time
    except Exception as e:
        print(e)
        travel_time = None
    
    return travel_time

#async def travel_time_process(row:pd.Series, sdk: TravelTimeSdk, transport_type:Transportation) -> int:
    # destination = StationGeography(row['planned_ada_station_name'], 
    #                                    row['planned_ada_complex_latitude'], 
    #                                    row['planned_ada_complex_longitude'])
    # origin = StationGeography(
    # row['existing_ada_stop_name'],
    # row['existing_ada_complex_latitude'],
    # row['existing_ada_complex_longitude']
    # )
    #     # Check if an event loop is already running
    # if not asyncio.get_event_loop().is_running():
    #     results = asyncio.run(time_filter_wrapper(sdk, origin, destination, transport_type))
    # else:
    #     # Use `await` if you're in an interactive environment like Jupyter
    #     results = await time_filter_wrapper(sdk, origin, destination, transport_type)

async def process_all_rows(sdk:TravelTimeSdk, df: pd.DataFrame):
    """
    New attempt at running all rows in a dataframe

    sdk: Instance of TravelTimeSdk
    df: Dataframe in the specific latlong format from the data model
    """

    for index, row in df.iterrows():
        destination = StationGeography(row['planned_ada_station_name'], 
                                   row['planned_ada_complex_latitude'], 
                                   row['planned_ada_complex_longitude'])
        origin = StationGeography(row['existing_ada_stop_name'],
                                  row['existing_ada_complex_latitude'],
                                  row['existing_ada_complex_longitude'])
        walking_time_sec = await time_filter_wrapper(sdk, origin, destination, Walking())
        df.at[index, 'walking_time_sec'] = walking_time_sec

    return df

In [12]:
sdk = TravelTimeSdk(app_id="87ebd830",
                    api_key="a084a6856d76b7580d8390859bff3ac8")

new_df = asyncio.run(process_all_rows(sdk, df_latlongs.head(n=3)))


RuntimeError: asyncio.run() cannot be called from a running event loop